# import

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pdb
import time
from pprint import pprint
import requests
import json

import pandas as pd
from dotenv import load_dotenv, find_dotenv
from tqdm import tqdm

In [3]:
from IPython.display import Markdown, display

In [4]:
# Model
load_dotenv(find_dotenv()) # read local .env file
assert os.environ["JIGSAW_API_KEY"]  != ''
assert os.environ["JIGSAW_PUBLIC_KEY"]  != ''
assert os.environ["GROQ_API_KEY"]  != ''

# Jigsaw

In [4]:
JIGSAW_API_KEY = os.environ["JIGSAW_API_KEY"]

In [5]:
JIGSAW_PUBLIC_KEY = os.environ["JIGSAW_PUBLIC_KEY"]

## upload

In [6]:
# Define the endpoint and the API key
endpoint = "https://api.jigsawstack.com/v1/store/file?key=gettysburg.wav&overwrite=true"
print(endpoint)

# Define the headers
headers = {
    "Content-Type": "audio/wav",
    "x-api-key": JIGSAW_API_KEY,
}

https://api.jigsawstack.com/v1/store/file?key=gettysburg.wav&overwrite=true


In [7]:
%%time
# Define the body of the request
body = open('data/gettysburg.wav', 'rb')

# Make the POST request
response = requests.post(endpoint,
                         headers=headers,
                         data=body
                        )

# Parse the JSON response
data = response.json()

# Print the result
print(data)

{'success': True, 'url': 'https://api.jigsawstack.com/v1/store/file/gettysburg.wav', 'key': 'gettysburg.wav'}
CPU times: user 45.2 ms, sys: 63.1 ms, total: 108 ms
Wall time: 4.62 s


## transcribe

In [8]:
# Define the endpoint and the API key
endpoint = "https://api.jigsawstack.com/v1/ai/transcribe"

# Define the headers
headers = {
    "Content-Type": "application/json",
    "x-api-key": JIGSAW_API_KEY,
}

# Define the body of the request
body = {
    "file_store_key": "gettysburg.wav",
}

In [9]:
print(endpoint)
print(headers)
print(body)

https://api.jigsawstack.com/v1/ai/transcribe
{'Content-Type': 'application/json', 'x-api-key': 'sk_acd188330c1f9f68e2bc78bd43d8166965a8dad216eb01cb95a0a338b962ef30639891eb20133044ef621920a9f2dde49c00b5f0eedb6977219e1843f0d7688d024AFgZoUHfu6FCv1Hjg4'}
{'file_store_key': 'gettysburg.wav'}


In [10]:
%%time
# Make the POST request
response = requests.post(endpoint,
                         headers=headers,
                         data=json.dumps(body)
                        )

# Parse the JSON response
data = response.json()

# Print the result
print(data)

{'success': True, 'text': ' four score and seven years ago our fathers brought forth on this continent a new nation conceived in liberty and dedicated to the proposition that all men are created equal now we are engaged in a great civil war testing whether that nation or any nation so conceived and so dedicated can long endure', 'chunks': [{'timestamp': [0, 17.58], 'text': ' four score and seven years ago our fathers brought forth on this continent a new nation conceived in liberty and dedicated to the proposition that all men are created equal now we are engaged in a great civil war testing whether that nation or any nation so conceived and so dedicated can long endure'}]}
CPU times: user 11.4 ms, sys: 1.08 ms, total: 12.5 ms
Wall time: 37.6 s


In [11]:
%%time
# Define the endpoint and the API key
endpoint = "https://api.jigsawstack.com/v1/ai/transcribe"

# Define the headers
headers = {
    "Content-Type": "application/json",
    "x-api-key": JIGSAW_API_KEY,
}

# Define the body of the request
body = {
    "file_store_key": "gettysburg.wav",
}

# Make the POST request
response = requests.post(endpoint,
                         headers=headers,
                         data=json.dumps(body)
                        )

# Parse the JSON response
data = response.json()

# Print the result
print(data)

{'success': True, 'text': ' four score and seven years ago our fathers brought forth on this continent a new nation conceived in liberty and dedicated to the proposition that all men are created equal now we are engaged in a great civil war testing whether that nation or any nation so conceived and so dedicated can long endure', 'chunks': [{'timestamp': [0, 17.58], 'text': ' four score and seven years ago our fathers brought forth on this continent a new nation conceived in liberty and dedicated to the proposition that all men are created equal now we are engaged in a great civil war testing whether that nation or any nation so conceived and so dedicated can long endure'}]}
CPU times: user 6.02 ms, sys: 18.1 ms, total: 24.1 ms
Wall time: 6.99 s


# Groq (resume parser)

In [12]:
os.environ["GROQ_KEY"] = os.environ["GROQ_API_KEY"]

In [15]:
import PyPDF2

In [13]:
from splitResume import get_response

In [16]:
uploaded_file = "./data/JohnDoeResume.pdf"

pdfReader = PyPDF2.PdfReader(uploaded_file)
resume_text = "\n"
for pageObj in pdfReader.pages:
    current_text = pageObj.extract_text()
    resume_text += f"{current_text}\n"

response = get_response(resume_text)

In [18]:
display(Markdown(response))

Here is the resume split into different sections:

**Contact Information**

* Name: John Doe
* Address: Simei Street 1, (520133)
* Email: [John_Doe@hotmail.com](mailto:John_Doe@hotmail.com)
* LinkedIn: https://www.linkedin.com/in/john-doe-840510214

**Summary of Qualifications**

* Experienced in designing solutions for environmental problems
* Proficient in various modeling and drafting software
* Efficiently manages projects and collaborates with teams

**Education**

* Institution: Southeastern Louisiana University, Hammond, LA
* Degree: Bachelor of Science in Engineering Technology, Concentration: Electrical Energy Engineering
* Graduation Date: May 2024
* Cumulative GPA: 3.75/4.00
* Relevant Coursework:
	+ Introduction to Programming
	+ Electrical Circuits
	+ Electromagnetics
	+ Programming for Technologists

**Technical Proficiencies**

* Modeling Programs:
	+ ALGOR
	+ eQUEST
	+ EnergyPro (LEED project with a VRF system)
* Drafting Software:
	+ AutoCAD
	+ AutoCAD LT

**Experience**

* American Pollution Control Corp., Environmental Engineering Intern, Chalmette, LA (June 2023 - August 2023)
	+ Inspected sites and performed detailed monitoring of industrial pollution control measures
	+ Served on a committee dedicated to designing and implementing a new wastewater treatment system
	+ Investigated environmental projects onsite with a team of 3 engineers
* Cargill, Engineer Intern, Breaux Bridge, LA (May 2022 - August 2022)
	+ Researched building code items, materials, and similar building plans for 2 large commercial projects in New Orleans
	+ Reviewed building plans with engineering and design teams to evaluate for ADA compliance
	+ Collaborated with a team of 7 to research and identify suitable locations to install groundwater dams

**Project Experience**

* Group Project, Senior Design Course, Hammond, LA (January - March 2023)
	+ Collaborated with a 5-person team to develop an action plan for addressing societal, environmental, regulatory, and economic constraints related to a local wastewater project
	+ Researched client needs and developed a solutions-based layout to best suit functionality requirements
	+ Served as project leader by organizing team meetings, tracking progress, and providing a forum for discussion

**Memberships and Associations**

* The National Association of Environmental Professionals, Baton Rouge, LA (January 2020 - Present)
* Louisiana Association of Environmental Professionals (LAEP), Baton Rouge, LA (August 2020 - Present)

**Campus Involvement**

* Co-Captain, SLU Tennis Team, Hammond, LA (August 2020 - May 2023)
* Member, Beta Gamma Sigma, SLU, Hammond, LA (December 2019 - December 2020)

# Graph

In [38]:
from typing_extensions import TypedDict
from langgraph.graph import END, StateGraph
from PIL import Image
import io

In [39]:
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question about the candidate
        attachment: {'audio': path, 'resume': path}
        answer: Agent's answer to question
    """
    question : str
    attachment : dict
    answer : str
    # aggregate: Annotated[list, operator.add]

In [46]:
def _node_get_intent(state):
    state
    return state
def _node_resume_parser(state):
    resume_path = state['resume']
    print(f'Uploaded resume from {resume_path}')
    return state
def _node_speech_parser(state):
    speech_path = state['speech']
    print(f'Uploaded speech from {speech_path}')
    return state

def _edge_xxx1():
    return "upload_resume"
    return "upload_speech"
    return "get_summary"
    return "ask_question"

def _edge_xxx2():
    return "resume_parser"
    return "speech_parser"
    return "end"

def setup_agent():
    print('\n--- setup_agent')
    # langgraph.graph
    workflow = StateGraph(GraphState)

    ## Define the nodes
    workflow.add_node("NODE_get_intent", _node_get_intent)
    workflow.add_node("NODE_resume_parser", _node_resume_parser)
    workflow.add_node("NODE_speech_parser", _node_speech_parser)
    workflow.add_node("NODE_store_data", _node_xxx)
    workflow.add_node("NODE_summary", _node_xxx)
    workflow.add_node("NODE_qna", _node_xxx)

    ## Add Edges
    workflow.set_entry_point("NODE_intent")
    workflow.add_conditional_edges(
        "NODE_intent",
        _edge_xxx1,
        {
            "upload_resume": "NODE_resume_parser",
            "upload_speech": "NODE_speech_parser",
            "get_summary": "NODE_summary",
            "ask_question": "NODE_qna",
        },
    )

    workflow.add_edge("NODE_resume_parser", "NODE_store_data")
    workflow.add_edge("NODE_speech_parser", "NODE_store_data")
    ## To sink node
    workflow.add_edge("NODE_store_data", END)
    workflow.add_edge("NODE_summary", END)
    workflow.add_edge("NODE_qna", END)

    # Compile
    app = workflow.compile()

    # Show Image
    app_img = app.get_graph().draw_mermaid_png()
    image = Image.open(io.BytesIO(app_img))
    image.save("graph.png")

    return app

app = setup_agent()
print('done')


--- setup_agent
done


# Call

In [5]:
from typing_extensions import TypedDict
from langgraph.graph import END, StateGraph
from PIL import Image
import io

In [6]:
from utils import RecruiterAgent

In [7]:
recruiter_agent = RecruiterAgent()


--- load_llm_model
	Using llama3-8b-8192

--- setup_chains
	 _chain_resume_parser
	 _chain_speech_parser
	 _chain_question_answering


In [10]:
app = recruiter_agent.setup_agent()
recruiter_agent.verbose = True


--- setup_agent


In [11]:
input_data = {
    'question': 'hi'
}
app.invoke(input_data)

	 intent = ask_question
---[EDGE] _edge_routing_to_agents---
self.state = {}


{'question': 'hi',
 'intent': 'ask_question',
 'attachment': None,
 'answer': '\nHello! It seems like you said "hi" without any specific question or topic. Is there something on your mind that you\'d like to talk about or ask? I\'m here to listen and help if I can!'}

In [12]:
%%time
# recruiter_agent = RecruiterAgent()
# app = recruiter_agent.setup_agent()
# recruiter_agent.verbose = True
input_data = {    'attachment': './data/JohnDoeResume.pdf'}
resume_output = app.invoke(input_data)
resume_output

	 intent = upload_resume
---[EDGE] _edge_routing_to_agents---
	Uploaded resume from ./data/JohnDoeResume.pdf
	endpoint = https://api.jigsawstack.com/v1/store/file?key=JohnDoeResume.pdf&overwrite=true
	data = {'success': True, 'url': 'https://api.jigsawstack.com/v1/store/file/JohnDoeResume.pdf', 'key': 'JohnDoeResume.pdf'}
	resume_content = 
Based on the provided resume, I have categorized the information into the following sections:

**Summary**

* Summary of qualifications, highlighting John Doe's experience in designing solutions for environmental problems, proficiency in modeling and drafting software, and ability to manage projects and collaborate with teams.

**Education**

* Bachelor of Science in Engineering Technology, Southeastern Louisiana University, May 2024
	+ Concentration: Electrical Energy Engineering
	+ Cumulative GPA: 3.75/4.00
	+ Relevant Coursework: Introduction to Programming, Electrical Circuits, Electromagnetics, Programming for Technologists

**Technical Profici

{'question': None,
 'intent': 'upload_resume',
 'attachment': './data/JohnDoeResume.pdf',
 'resume_key': 'JohnDoeResume.pdf',
 'resume_content': "Based on the provided resume, I have categorized the information into the following sections:\n\n**Summary**\n\n* Summary of qualifications, highlighting John Doe's experience in designing solutions for environmental problems, proficiency in modeling and drafting software, and ability to manage projects and collaborate with teams.\n\n**Education**\n\n* Bachelor of Science in Engineering Technology, Southeastern Louisiana University, May 2024\n\t+ Concentration: Electrical Energy Engineering\n\t+ Cumulative GPA: 3.75/4.00\n\t+ Relevant Coursework: Introduction to Programming, Electrical Circuits, Electromagnetics, Programming for Technologists\n\n**Technical Proficiencies**\n\n* Modeling Programs: ALGOR, eQUEST, EnergyPro\n* Drafting Software: AutoCAD, AutoCAD LT\n\n**Relevant Experience**\n\n* American Pollution Control Corp., June 2023 – Aug

In [13]:
%%time
# recruiter_agent = RecruiterAgent()
# app = recruiter_agent.setup_agent()
# recruiter_agent.verbose = True
input_data = {    'question': 'what is the educational history ?'}
app.invoke(input_data)

	 intent = ask_question
---[EDGE] _edge_routing_to_agents---
self.state = {'resume_content': "Based on the provided resume, I have categorized the information into the following sections:\n\n**Summary**\n\n* Summary of qualifications, highlighting John Doe's experience in designing solutions for environmental problems, proficiency in modeling and drafting software, and ability to manage projects and collaborate with teams.\n\n**Education**\n\n* Bachelor of Science in Engineering Technology, Southeastern Louisiana University, May 2024\n\t+ Concentration: Electrical Energy Engineering\n\t+ Cumulative GPA: 3.75/4.00\n\t+ Relevant Coursework: Introduction to Programming, Electrical Circuits, Electromagnetics, Programming for Technologists\n\n**Technical Proficiencies**\n\n* Modeling Programs: ALGOR, eQUEST, EnergyPro\n* Drafting Software: AutoCAD, AutoCAD LT\n\n**Relevant Experience**\n\n* American Pollution Control Corp., June 2023 – August 2023\n\t+ Environmental Engineering Intern\n\t+ 

{'question': 'what is the educational history ?',
 'intent': 'ask_question',
 'attachment': None,
 'answer': '\n\nJohn Doe has a Bachelor of Science in Engineering Technology from Southeastern Louisiana University, with a concentration in Electrical Energy Engineering. He achieved a cumulative GPA of 3.75/4.00 and completed relevant coursework in programming, electrical circuits, electromagnetics, and programming for technologists.\n\n'}

In [14]:
%%time
# recruiter_agent = RecruiterAgent()
# app = recruiter_agent.setup_agent()
# recruiter_agent.verbose = True
input_data = { 'attachment': './data/gettysburg.wav' }
speech_output = app.invoke(input_data)
speech_output

	 intent = upload_speech
---[EDGE] _edge_routing_to_agents---
	Uploaded speech from ./data/gettysburg.wav
	endpoint = https://api.jigsawstack.com/v1/store/file?key=gettysburg.wav&overwrite=true
	data = {'success': True, 'url': 'https://api.jigsawstack.com/v1/store/file/gettysburg.wav', 'key': 'gettysburg.wav'}
	endpoint = https://api.jigsawstack.com/v1/ai/transcribe
	body = {'file_store_key': 'gettysburg.wav'}
	data = {'success': True, 'text': ' four score and seven years ago our fathers brought forth on this continent a new nation conceived in liberty and dedicated to the proposition that all men are created equal now we are engaged in a great civil war testing whether that nation or any nation so conceived and so dedicated can long endure', 'chunks': [{'timestamp': [0, 17.58], 'text': ' four score and seven years ago our fathers brought forth on this continent a new nation conceived in liberty and dedicated to the proposition that all men are created equal now we are engaged in a gre

{'question': None,
 'intent': 'upload_speech',
 'attachment': './data/gettysburg.wav',
 'speech_transcribed': ' four score and seven years ago our fathers brought forth on this continent a new nation conceived in liberty and dedicated to the proposition that all men are created equal now we are engaged in a great civil war testing whether that nation or any nation so conceived and so dedicated can long endure',
 'speech_keypoints': '\n{\n  "Name": null,\n  "Gender": null,\n  "Languages": null,\n  "Country": null,\n  "Work Pass Type": null,\n  "Nationality": null,\n  "Notice Period": null,\n  "Reason for looking out": null,\n  "Looking for in next move": null,\n  "Other Interviews in pipeline": null,\n  "Current Salary": null,\n  "Expected Salary": null,\n  "Identity": null,\n  "Open to Consulting model": null,\n  "Current Annual Leave days": null,\n  "Worked for Client Before or no": null,\n  "Hiring for other roles?": null,\n  "General Notes": null\n}\n',
 'answer': ''}

In [16]:
%%time
# recruiter_agent = RecruiterAgent()
# app = recruiter_agent.setup_agent()
# recruiter_agent.verbose = True
input_data = {    'question': 'what is the latest job history ?'}
app.invoke(input_data)

	 intent = ask_question
---[EDGE] _edge_routing_to_agents---
self.state = {'resume_content': "Based on the provided resume, I have categorized the information into the following sections:\n\n**Summary**\n\n* Summary of qualifications, highlighting John Doe's experience in designing solutions for environmental problems, proficiency in modeling and drafting software, and ability to manage projects and collaborate with teams.\n\n**Education**\n\n* Bachelor of Science in Engineering Technology, Southeastern Louisiana University, May 2024\n\t+ Concentration: Electrical Energy Engineering\n\t+ Cumulative GPA: 3.75/4.00\n\t+ Relevant Coursework: Introduction to Programming, Electrical Circuits, Electromagnetics, Programming for Technologists\n\n**Technical Proficiencies**\n\n* Modeling Programs: ALGOR, eQUEST, EnergyPro\n* Drafting Software: AutoCAD, AutoCAD LT\n\n**Relevant Experience**\n\n* American Pollution Control Corp., June 2023 – August 2023\n\t+ Environmental Engineering Intern\n\t+ 

{'question': 'what is the latest job history ?',
 'intent': 'ask_question',
 'attachment': None,
 'answer': 'The latest job history of John Doe is his experience as an Environmental Engineering Intern at American Pollution Control Corp. from June 2023 to August 2023.'}

In [32]:
%%time
recruiter_agent = RecruiterAgent()
app = recruiter_agent.setup_agent()
recruiter_agent.verbose = True

resume_content = resume_output['resume_content']
speech_keypoints = speech_output['speech_keypoints']


--- load_llm_model
	Using llama3-8b-8192

--- setup_chains
	 _chain_COT
	 _chain_COT

--- setup_agent
CPU times: user 115 ms, sys: 2.22 ms, total: 117 ms
Wall time: 116 ms


In [46]:
input_data = { 'question': 'get_summary', 'resume_content': resume_content, 'speech_keypoints': json.loads(speech_keypoints), }
summary_output = app.invoke(input_data)

	 intent = get_summary
---[EDGE] _edge_routing_to_agents---


In [47]:
display(Markdown(summary_output['answer']))

Based on the provided resume, I have categorized the information into the following sections:

**Summary**

* Experienced in designing solutions for environmental problems that require integration of environmental and human health constraints compatible with economic growth, sustainable development, and ethical standards.
* Proficient with a variety of modeling and drafting software including AutoCAD and ALGOR.
* Efficiently manage projects and collaborate with teams to accelerate the overarching goals of the company.

**Education**

* Southeastern Louisiana University - Hammond, LA
	+ Bachelor of Science in Engineering Technology, Concentration: Electrical Energy Engineering
	+ Cumulative GPA: 3.75/4.00
	+ Relevant Coursework: Introduction to Programming, Electrical Circuits, Electromagnetics, Programming for Technologists

**Technical Proficiencies**

* Modeling Programs: ALGOR, eQUEST, and EnergyPro (LEED project with a VRF system)
* Drafting Software: AutoCAD, AutoCAD LT

**Relevant Experience**

* American Pollution Control Corp., June 2023 - August 2023
	+ Environmental Engineering Intern - Chalmette, LA
		- Inspected sites and performed detailed monitoring of industrial pollution control measures with an environmental engineer based on state and local regulations.
		- Served on a committee dedicated to designing and implementing a new wastewater treatment system.
		- Investigated environmental projects onsite with a team of 3 engineers.
* Cargill, May 2022 - August 2022
	+ Engineer Intern - Breaux Bridge, LA
		- Researched building code items, materials, and similar building plans for 2 large commercial projects in New Orleans.
		- Reviewed building plans with engineering and design teams to evaluate for ADA compliance.
		- Collaborated with a team of 7 to research and identify suitable locations to install groundwater dams.

**Relevant Project Experience**

* Group Project, Senior Design Course - Hammond, LA, January - March 2023
	+ Collaborated with a 5-person team tasked with developing an action plan for addressing societal, environmental, regulatory, and economic constraints related to a local wastewater project.
	+ Researched client needs and developed a solutions-based layout to best suit functionality requirements.
	+ Served as project leader by organizing team meetings, tracking progress, and providing a forum for discussion.

**Memberships and Associations**

* The National Association of Environmental Professionals - Baton Rouge, LA, January 2020 - Present
* Louisiana Association of Environmental Professionals (LAEP) - Baton Rouge, LA, August 2020 - Present

**Campus Involvement**

* Co-Captain, SLU Tennis Team - Hammond, LA, August 2020 - May 2023
	+ Instrumental in team motivation, encouragement, and success.
* Member, Beta Gamma Sigma, SLU - Hammond, LA, December 2019 - December 2020
	+ International Honor Society


**Conversation**
* Name
	+ None
* Gender
	+ None
* Languages
	+ None
* Country
	+ None
* Work Pass Type
	+ None
* Nationality
	+ None
* Notice Period
	+ None
* Reason for looking out
	+ None
* Looking for in next move
	+ None
* Other Interviews in pipeline
	+ None
* Current Salary
	+ None
* Expected Salary
	+ None
* Identity
	+ None
* Open to Consulting model
	+ None
* Current Annual Leave days
	+ None
* Worked for Client Before or no
	+ None
* Hiring for other roles?
	+ None
* General Notes
	+ None

In [69]:
%%time
recruiter_agent = RecruiterAgent()
app = recruiter_agent.setup_agent()
recruiter_agent.verbose = True

resume_content = resume_output['resume_content']
speech_keypoints = speech_output['speech_keypoints']


--- load_llm_model
	Using llama3-8b-8192

--- setup_chains
	 _chain_resume_parser
	 _chain_speech_parser
	 _chain_question_answering

--- setup_agent
CPU times: user 27 ms, sys: 2.02 ms, total: 29 ms
Wall time: 28.4 ms


In [70]:
input_data = {'question': 'what is the education of John Doe ?',
              'resume_content': resume_content,
              'speech_keypoints': json.loads(speech_keypoints),
             }
answer_output = app.invoke(input_data)

	 intent = ask_question
---[EDGE] _edge_routing_to_agents---


In [71]:
print(answer_output['answer'])


The individual's education is a Bachelor of Science in Engineering Technology, Concentration: Electrical Energy Engineering from Southeastern Louisiana University with a cumulative GPA of 3.75/4.00.


## plot graph

In [45]:
app_img = app.get_graph().draw_mermaid_png()
image = Image.open(io.BytesIO(app_img))
image.save("graph.png")

# debug